<a href="https://colab.research.google.com/github/freehtet/NLP/blob/main/Sentiment_Analysis_for_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import nltk
import nltk.stem.porter
from sklearn.preprocessing import LabelEncoder
import torch
import re
import evaluate
import datasets
import transformers

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')

In [ ]:
df.shape

(23486, 11)

In [ ]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df = df[['Review Text', 'Rating']]

In [ ]:
df.columns = ["text", "label"]

In [ ]:
df.head()

,text,label
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5


In [ ]:
df['text'] = df['text'].astype(str)

<ipython-input-12-e72252b4a215>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)


Text Cleaning

In [ ]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = text.strip()  # Remove leading/trailing spaces
    return text

In [ ]:
df['clean_text'] = df['text'].apply(clean_text)

In [ ]:
df.head()

,text,label,clean_text
0,Absolutely wonderful - silky and sexy and comf...,4,absolutely wonderful silky and sexy and comfo...
1,Love this dress! it's sooo pretty. i happene...,5,love this dress its sooo pretty i happened t...
2,I had such high hopes for this dress and reall...,3,i had such high hopes for this dress and reall...
3,"I love, love, love this jumpsuit. it's fun, fl...",5,i love love love this jumpsuit its fun flirty ...
4,This shirt is very flattering to all due to th...,5,this shirt is very flattering to all due to th...


Tokenization

In [ ]:
def custom_tokenizer(text):
    return nltk.word_tokenize(text)

# TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', tokenizer=custom_tokenizer)
tfidf_features = tfidf_vectorizer.fit_transform(df['clean_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Train-Test-Split

In [ ]:
X = tfidf_features
y = df['label']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

Validation split

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)

# Logistic Regression

In [ ]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_val_preds = lr_model.predict(X_val)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
accuracy = accuracy_score(y_val, lr_val_preds)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.6346604215456675


# Support Vector Machine

In [ ]:
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)
svm_val_preds = svm_model.predict(X_val)

In [ ]:
accuracy = accuracy_score(y_val, svm_val_preds)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.6387055567383436


# Random Forest Classifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_val_preds = rf_model.predict(X_val)

In [ ]:
accuracy = accuracy_score(y_val, rf_val_preds)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.5795188418139238


Transformer

In [ ]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [ ]:
df.head()

,text,label,clean_text
0,Absolutely wonderful - silky and sexy and comf...,3,absolutely wonderful silky and sexy and comfo...
1,Love this dress! it's sooo pretty. i happene...,4,love this dress its sooo pretty i happened t...
2,I had such high hopes for this dress and reall...,2,i had such high hopes for this dress and reall...
3,"I love, love, love this jumpsuit. it's fun, fl...",4,i love love love this jumpsuit its fun flirty ...
4,This shirt is very flattering to all due to th...,4,this shirt is very flattering to all due to th...


In [ ]:
df_train, df_test = train_test_split(df.sample(1000,random_state=42), test_size=0.2, random_state=42)

In [ ]:
model_name = "distilroberta-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

df_train_encoded = datasets.Dataset.from_pandas(df_train).map(preprocess_function, batched=True)
df_test_encoded = datasets.Dataset.from_pandas(df_test).map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=df['label'].unique().shape[0])

training_args = transformers.TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    report_to="none",
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train_encoded,
    eval_dataset=df_test_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("./results/" + model_name)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-31-f866744ae6a6>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.144519,0.540000
2,No log,0.987152,0.575000
3,No log,0.848753,0.650000
4,No log,0.955486,0.630000
5,No log,0.905989,0.670000
6,No log,0.885415,0.660000
7,No log,0.923331,0.650000
8,No log,0.976082,0.665000
9,No log,0.970554,0.670000
10,0.707200,0.977949,0.680000


In [ ]:
trainer.evaluate()

{'eval_loss': 0.9779493808746338,
 'eval_accuracy': 0.68,
 'eval_runtime': 42.8201,
 'eval_samples_per_second': 4.671,
 'eval_steps_per_second': 0.304,
 'epoch': 10.0}

In [ ]:
new_reviews = ["Amazing quality and fast shipping!", "Horrible product, broke within a day."]
inputs = tokenizer(new_reviews, padding=True, truncation=True, return_tensors="pt")
outputs = model(**inputs)
predicted_ratings = torch.argmax(outputs.logits, dim=1) + 1  # Ratings start from 1
print(predicted_ratings)

tensor([5, 2])
